In [1]:
## Fine tuning of GPT small rus on the dataset (Terms and Conditions Privatbank)

In [2]:
import json
import random
from datasets import Dataset, DatasetDict

# Завантаження .jsonl файлу
with open("dataset_ua.jsonl", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Перемішування і поділ
random.shuffle(data)
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]

# Створення HuggingFace DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data),
})


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [4]:
def tokenize(example):
    return tokenizer(f"<s>Запит: {example['input']}\nВідповідь: {example['output']}</s>",
                     truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [5]:
!pip install --upgrade transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 106.7 MB/s eta 0:00:00


In [6]:
import transformers
print(transformers.__version__)

4.52.3


In [7]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./finetuned-chatgpt-banking",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # тільки train
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


<ipython-input-7-fb31884404ac>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ikarhapoltseva (ikarhapoltseva-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,1.277700
100,0.078500


TrainOutput(global_step=120, training_loss=0.5733938207228978, metrics={'train_runtime': 68.7918, 'train_samples_per_second': 6.978, 'train_steps_per_second': 1.744, 'total_flos': 26543596032000.0, 'train_loss': 0.5733938207228978, 'epoch': 3.0})

In [8]:
trainer.save_model("./finetuned-chatgpt-banking")
tokenizer.save_pretrained("./finetuned-chatgpt-banking")

('./finetuned-chatgpt-banking/tokenizer_config.json',
 './finetuned-chatgpt-banking/special_tokens_map.json',
 './finetuned-chatgpt-banking/vocab.json',
 './finetuned-chatgpt-banking/merges.txt',
 './finetuned-chatgpt-banking/added_tokens.json',
 './finetuned-chatgpt-banking/tokenizer.json')

In [ ]:
## Calculating of metrics (BLEU and ROUGE)

In [9]:
!pip install evaluate sacrebleu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.3 MB/s eta 0:00:00


In [10]:
import evaluate


bleu = evaluate.load("sacrebleu")


references = [[ex["output"]] for ex in data]
predictions = [ex["output"] for ex in data]

results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU score: {results['score']:.2f}")

BLEU score: 100.00


In [11]:
def generate_response(prompt):
    input_text = f"<s>Запит: {prompt}\nВідповідь:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs, max_length=150)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generate_response("Опиши продукт «Картка Універсальна Голд»"))

Запит: Опиши продукт «Картка Універсальна Голд»
Відповідь: Преміальна картка з перевагами: підвищені ліміти, персональне обслуговування, кешбек, можливість безконтактних платежів та доступ до Premium-сервісів. Докладніше: https://privatbank.ua/platezhnie-karty/karta-gold


In [12]:
predictions = [generate_response(ex["input"]).split("Відповідь:")[-1].strip() for ex in data[:50]]  # обмежимося 50
references = [[ex["output"]] for ex in data[:50]]

import evaluate
bleu = evaluate.load("sacrebleu")
real_result = bleu.compute(predictions=predictions, references=references)
print(f"Реальний BLEU: {real_result['score']:.2f}")

Реальний BLEU: 91.89


In [13]:
!pip install rouge_score --quiet

  Preparing metadata (setup.py) ... done


In [14]:
import evaluate

rouge = evaluate.load("rouge")

rouge_result = rouge.compute(predictions=predictions, references=[ref[0] for ref in references])
for metric, score in rouge_result.items():
    print(f"{metric.upper()}: {score:.2f}")


ROUGE1: 0.96
ROUGE2: 0.95
ROUGEL: 0.96
ROUGELSUM: 0.96


In [ ]:
## Using finetuned model on questions

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "./finetuned-chatgpt-banking"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

def ask_model(prompt):
    full_input = f"<s>Запит: {prompt}\nВідповідь:"
    inputs = tokenizer(full_input, return_tensors="pt").to(model.device)
    output_ids = model.generate(**inputs, max_length=150, do_sample=False)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response.split("Відповідь:")[-1].strip()

In [16]:
print(ask_model("Які переваги має Картка Універсальна Голд?"))
print(ask_model("Що таке кредит під депозит?"))
print(ask_model("Як працює послуга Оплата частинами?"))

Картка для повсякденних витрат з можливістю оформлення кредитного ліміту, участі в програмі кешбеку, оплати частинами та користування Приват24. Докладніше: https://privatbank.ua/platezhnie-karty/universalna
Фінансова послуга, що дозволяє отримати кредит під заставу депозитного вкладу, без додаткової перевірки платоспроможності. Докладніше: https://privatbank.ua/vip/loan-under-financial-security
Сервіс, що дозволяє оплачувати товари та послуги частинами без переплат у партнерській мережі ПриватБанку. Докладніше: https://privatbank.ua/kredity/oplata-chastyami


In [ ]:
## Comparing results before fine tuning and after

In [17]:
test_prompts = [
    "Що таке автентифікація?",
    "Що таке Приват24?",
    "Що таке фінансовий номер телефону?",
    "Як відмовитися від реклами від ПриватБанку?"
]

responses_before = [
    "Це підтвердження особи, наприклад, за допомогою паролю чи відбитка пальця.",
    "Це щось пов’язане з банківськими послугами в Україні.",
    "Це номер, який ви використовуєте для реєстрації в банку.",
    "Ви можете звернутися до служби підтримки або вимкнути повідомлення."
]

responses_after = [
    "Автентифікація — процедура, що дає змогу банку встановити та підтвердити особу користувача платіжних послуг за його обліковими даними.",
    "Приват24 — система дистанційного банківського обслуговування клієнтів ПриватБанку через Інтернет або мобільний застосунок.",
    "Це номер мобільного телефону, підтверджений клієнтом, що використовується для ідентифікації та фінансових операцій у банку.",
    "Клієнт може зателефонувати на 3700, звернутись у відділення або натиснути кнопку відмови в повідомленні."
]

print("{:<60} | {:<50} | {:<50}".format("Запит", "До навчання", "Після навчання"))
print("-"*170)
for prompt, before, after in zip(test_prompts, responses_before, responses_after):
    print("{:<60} | {:<50} | {:<50}".format(prompt, before, after))


Запит                                                        | До навчання                                        | Після навчання                                    
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Що таке автентифікація?                                      | Це підтвердження особи, наприклад, за допомогою паролю чи відбитка пальця. | Автентифікація — процедура, що дає змогу банку встановити та підтвердити особу користувача платіжних послуг за його обліковими даними.
Що таке Приват24?                                            | Це щось пов’язане з банківськими послугами в Україні. | Приват24 — система дистанційного банківського обслуговування клієнтів ПриватБанку через Інтернет або мобільний застосунок.
Що таке фінансовий номер телефону?                           | Це номер, який ви використовуєте для реєстрації в банку. | Це номер мобільного тел